In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [4]:
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.6 MB/s eta 0:00:00


In [9]:
!pip install --upgrade transformers spacy-transformers

In [ ]:
!pip install -U spacy-transformers

In [ ]:
!pip install -U spacy

In [11]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [14]:
spacy.__version__

'3.6.1'

In [ ]:
!nvidia-smi

Sun Oct  1 07:25:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
# Load the annotated data from a JSON file
import json
cv_data = json.load(open('/content/drive/MyDrive/Colab Notebooks/Resume-Parser/CV-Parser-Spacy3/dataset/dataset1014.json', 'r'))

In [17]:
len(cv_data)

1014

In [ ]:
!python -m spacy init fill-config '/content/drive/MyDrive/Colab Notebooks/Resume-Parser/CV-Parser-Spacy3/config/base_config.cfg' '/content/drive/MyDrive/Colab Notebooks/Resume-Parser/CV-Parser-Spacy3/config/config.cfg'


2023-10-01 14:10:06.366432: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Colab
Notebooks/Resume-Parser/CV-Parser-Spacy3/config/config1.cfg
You can now add your data and train your pipeline:
python -m spacy train config1.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True   # if entities are overlapping
          break
      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db


In [ ]:
!pip install scikit-learn

In [ ]:
# Split the annotated data into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [ ]:
# Display the number of items in the training and testing sets
len(train), len(test)

(709, 305)

In [ ]:
# Open a file to log errors during annotation processing
file = open('/content/drive/MyDrive/Colab Notebooks/Resume-Parser/CV-Parser-Spacy3/model-data/error_file.txt','w')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('/content/drive/MyDrive/Colab Notebooks/Resume-Parser/CV-Parser-Spacy3/model-data/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('/content/drive/MyDrive/Colab Notebooks/Resume-Parser/CV-Parser-Spacy3/model-data/test_data.spacy')

# Close the error log file
file.close()

100%|██████████| 305/305 [00:02<00:00, 128.62it/s]


In [ ]:
len(db.tokens)

305

In [18]:
!python -m spacy train '/content/drive/MyDrive/Colab Notebooks/Resume-Parser/CV-Parser-Spacy3/config/config.cfg' --output '/content/drive/MyDrive/Colab Notebooks/Resume-Parser/CV-Parser-Spacy3/model-output' --paths.train '/content/drive/MyDrive/Colab Notebooks/Resume-Parser/CV-Parser-Spacy3/model-data/train_data.spacy' --paths.dev '/content/drive/MyDrive/Colab Notebooks/Resume-Parser/CV-Parser-Spacy3/model-data/test_data.spacy' --gpu-id 0


2023-10-20 11:00:41.308644: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: /content/drive/MyDrive/Colab
Notebooks/Resume-Parser/CV-Parser-Spacy3/model-output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0        5367.85   1213.60    0.06    0.04    0.20    0.00
  3     

In [ ]:
nlp = spacy.load('/content/drive/MyDrive/Colab Notebooks/Resume-Parser/CV-Parser-Spacy3/model-output/model-last')

In [ ]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 53.4 MB/s eta 0:00:00


In [ ]:
import sys, fitz

In [ ]:
fname = '/content/drive/MyDrive/Colab Notebooks/Resume-Parser/CV-Parser-Spacy3/test-files/Shreya Bhandari.pdf'
doc = fitz.open(fname)

text = " "
for page in doc:
  text = text + str(page.get_text())

text = text.strip()

text = ' '.join(text.split())

text

'Contact www.linkedin.com/in/shreya- bhandari-a0094816b (LinkedIn) shreyaabh.pythonanywhere.com/ (Portfolio) Top Skills Writing Python (Programming Language) Django Languages Nepali (Native or Bilingual) English Hindi Shreya Bhandari Python Developer|AI/ML enthusiast|Graphic Designer Kathmandu Experience Danson Solutions Business Analyst April 2022 - Present (4 months) Girls in Tech - Nepal 7 months Event Team Ambassador April 2022 - Present (4 months) Kathmandu, Bāgmatī, Nepal Support Squad Ambassador January 2022 - April 2022 (4 months) Mystery Gift Nepal Business Owner February 2022 - Present (6 months) Kathmandu, Bāgmatī, Nepal DansonTraining Project Lead June 2021 - Present (1 year 2 months) United States Finnove Technologies 6 months Associate Software Engineer November 2021 - November 2021 (1 month) Kathmandu, Bāgmatī, Nepal Python Developer June 2021 - November 2021 (6 months) Kathmandu, Bāgmatī, Nepal Danson Solutions Data Science Intern Page 1 of 2 August 2021 - October 2021 

In [ ]:
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, "---", ent.label_)

www.linkedin.com/in/shreya- bhandari-a0094816b --- LINKEDIN LINK
Python --- SKILLS
Django --- SKILLS
Nepali (Native or Bilingual) --- LANGUAGE
English --- SKILLS
Hindi --- SKILLS
Shreya Bhandari --- NAME
4 months --- YEARS OF EXPERIENCE
4 months --- YEARS OF EXPERIENCE
4 months --- YEARS OF EXPERIENCE
6 months --- YEARS OF EXPERIENCE
1 year 2 months --- YEARS OF EXPERIENCE
Finnove Technologies --- COMPANIES WORKED AT
Associate Software Engineer --- WORKED AS
1 month --- YEARS OF EXPERIENCE
6 months --- YEARS OF EXPERIENCE
3 months --- YEARS OF EXPERIENCE
5 months --- YEARS OF EXPERIENCE
Content Writer --- WORKED AS
1 year 1 month --- YEARS OF EXPERIENCE
Adve Content Writer --- WORKED AS
2 years 10 months --- YEARS OF EXPERIENCE
Nepal College of Information Technology --- COLLEGE NAME
